## Precinct 21 voters maps

My goal was to analyze the voters in precinct 21: specifically, what addresses don't have recorded votes.

I downloaded some data from the [Durham ArcGIS server](http://gisweb2.durhamnc.gov/ArcGIS/rest/services/PublicWorks/PublicWorks/MapServer).

Here is an example of the fields that each record has: [record 1](http://gisweb2.durhamnc.gov/ArcGIS/rest/services/PublicWorks/PublicWorks/MapServer/89/1)

Note: I use the form `test -f file || get file` to prevent re-running commands. You can delete the files to redownload.

In [13]:
PRECINCT = '21'
%env PRECINCT=$PRECINCT

env: PRECINCT=21


In [4]:
!test -f Precincts.shp || ( wget https://s3.amazonaws.com/dl.ncsbe.gov/PrecinctMaps/SBE_PRECINCTS_20161004.zip && unzip SBE_PRECINCTS_20161004.zip )

In [15]:
!test -f precinct-$PRECINCT.shp || ogr2ogr -sql "select * from Precincts where COUNTY_NAM='DURHAM' and PREC_ID='$PRECINCT'" -t_srs EPSG:4326 precinct-$PRECINCT.shp Precincts.shp

In [16]:
!ogr2ogr -f geojson precinct-$PRECINCT.geojson precinct-$PRECINCT.shp

After I got the boundary of the precinct voting disrict above, I fetched all the property boundaries from the Durham GIS server.

There are a lot of properties that are not residential, so I filter these out too:

In [317]:
!test -f landuse.json || curl "http://gisweb2.durhamnc.gov/arcgis/rest/services/PublicWorks/PublicWorks/MapServer/89/query?f=json&geometry=%7B%22ymax%22:832470.521274,%22xmax%22:2032594.388967,%22ymin%22:824305.219035,%22xmin%22:2027901.245138%7D&returnGeometry=false&outFields=SITE_ADDRE,OWNER_NAME,LANDUSE_DESC,LAND_USE" > landuse.json

In [318]:
!jq '.features|.[]|.attributes.LANDUSE_DESC' landuse.json | sort | uniq -c | sort -n | tail -20

   3 "CMNTY SVC/ SCHOOL        "
   3 "COM/ APARTMENT-GARDEN    "
   3 "COM/ CONV STORE W/ GAS   "
   3 "COM/ OFFICE BLDG         "
   3 "COM/ PARKING LOTS-SURFACE"
   3 "COM/ WHSE-STORAGE        "
   3 "VAC RES/ LOT-SML TR/SIDE "
   4 "COM/ 1-STY SM SGL USER   "
   4 "COM/ AUTO BODY-TIRE      "
   4 "COM/ FAST FOOD           "
   5 "COM/ MULTI-USE CAPABLE   "
   7 "COM/ CONVERTED RESIDENCE "
   7 "VAC RES/ LOT-SML TR/REAR "
   8 "COM/ 1-STY SM MULTI USER "
  15 "WILD/ PARKS & REC - CITY "
  19 "VACANT COMMERCIAL        "
  27 "VACANT COMMUNITY SERVICE "
  30 "VAC RES/ LOT-SML TRACT   "
  42 "RES/ 2-FAMILY            "
 769 "RES/ 1-FAMILY            "


In [319]:
!jq '.features|.[]|.attributes.LANDUSE_DESC' landuse.json | grep 'RES' | sort | uniq -c | sort -n

   1 "COM/ RESTAURANTS         "
   1 "RES/ RESIDENTIAL (UNDIFF)"
   1 "RES/ TOWNHOUSE W/ LAND   "
   1 "VAC RES/ DEVELOPER       "
   2 "RES/ MULTIPLE DWG'S      "
   3 "VAC RES/ LOT-SML TR/SIDE "
   7 "COM/ CONVERTED RESIDENCE "
   7 "VAC RES/ LOT-SML TR/REAR "
  30 "VAC RES/ LOT-SML TRACT   "
  42 "RES/ 2-FAMILY            "
 769 "RES/ 1-FAMILY            "


I used paw to mess with the HTTP params, but generally I just added the `"RES` filter above to my curl request. Note that ArcGIS does not create geojson out of the box. I used the ogr2ogr tool to convert it ([thanks SO](http://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson)).

But what I really want is everything within the precinct 21 area, so I need to make an intersection with the precinct boundary to filter out non-precinct properties.

The Durham GIS server has a limit to how many records you can download in one request. To avoid this exceededTransferLimit error, I batch the requests in chunks.

In [8]:
import json
import os
import requests
import geopandas as gpd

precinct = gpd.GeoDataFrame.from_file("precinct-21.shp")
bounds = precinct.bounds

all_lots_url_template = "http://gisweb2.durhamnc.gov/arcgis/rest/services/PublicWorks/PublicWorks/MapServer/89/query?f=json&geometry={},{},{},{}&returnGeometry=true&outFields=*&outSR=4326&inSR=4326"
all_lots_url = all_lots_url_template.format(bounds['minx'][0], bounds['miny'][0], bounds['maxx'][0], bounds['maxy'][0])

def strip_features(data):
    for row in data['features']:
        for k,v in row['attributes'].items():
            if isinstance(v, str):
                row['attributes'][k] = v.strip()
    return data

def download_arcgis(url, filename):   
    ids_request = requests.get("{}&returnIdsOnly=true".format(url))
    ids = ids_request.json()
    id_chunks = [ids['objectIds'][x:x+150] for x in range(0, len(ids['objectIds']), 150)]

    first_data = None
    for chunk in id_chunks:
        chunk_url = "{}&objectIds={}".format(url, ','.join(str(c) for c in chunk))
        data_request = requests.get(chunk_url)
        data = data_request.json()
        if first_data is None:
            first_data = strip_features(data)
        else:
            first_data['features'].extend(strip_features(data)['features'])

    jsonfile = '{}.json'.format(filename)
    with open(jsonfile,'w') as outfile:
        json.dump(first_data, outfile)        
        

if not os.path.exists('all_lots.json'):
    download_arcgis(all_lots_url, 'all_lots')
    !rm -f all_lots.geojson && ogr2ogr -f GeoJSON all_lots.geojson all_lots.json OGRGeoJSON
    !rm -f all_lots.shp && ogr2ogr all_lots.shp all_lots.geojson

<Response [200]>
Warning 6: Normalized/laundered field name: 'SplitTaxDis' to 'SplitTaxDi'
Warning 6: Normalized/laundered field name: 'SplitTaxDec' to 'SplitTaxDe'
Warning 6: Normalized/laundered field name: 'EXEMPT_CODE' to 'EXEMPT_COD'
Warning 6: Normalized/laundered field name: 'LANDUSE_DESC' to 'LANDUSE_DE'


In [10]:
lots = gpd.GeoDataFrame.from_file("all_lots.shp")

print('Properties from arcgis: {}'.format(len(lots)))

lots = lots.to_crs({'init': 'epsg:4326'})
precinct = precinct.to_crs({'init': 'epsg:4326'})

lots_in_precinct = gpd.sjoin(lots, precinct, op='within')

print('Properties in precinct 21: {}'.format(len(lots_in_precinct)))

lots_in_precinct.to_file('precinct-21-lots.shp')

Properties from arcgis: 1474
Properties in precinct 21: 1190


In [11]:
!rm -f precinct-21-lots.geojson && ogr2ogr -f geojson precinct-21-lots.geojson precinct-21-lots.shp

## Addresses in Princinct 21

In [12]:
import folium

map = folium.Map(location=[36.025, -78.899], tiles='CartoDB positron', zoom_start=15)
map.choropleth(geo_path='precinct-21-lots.geojson')
map.choropleth(geo_path='precinct-21.geojson', fill_opacity=0.2, line_opacity=0)
map